In [1]:
# Pickles for NSE

# STATUS: Completed
# Run-time: 1 hour 16 mins

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 3000, clientId=2)

In [2]:
%%time
import pandas as pd
import numpy as np
import itertools
import datetime
from math import sqrt, exp, log, erf
import os

#... Functions
#_____________

#... Error catching for list comprehension

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher
    Args: 
        (func) as the function
         (handle) as the lambda of function
         <*args | *kwargs> as arguments to the functions
    Outputs:
        output of the function | <np.nan> on error
    Usage:
        eggs = [1,3,0,3,2]
        [catch(lambda: 1/egg) for egg in eggs]'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan

def get_dte(dt):
    '''Gets days to expiry
    Arg: (dt) as day in string format yyyymmdd
    Returns: days to expiry as int'''
    return (util.parseIBDatetime(dt) - 
            datetime.datetime.now().date()).days
        
#...function to get historical data
def get_hist(contract, duration):
    '''Gets 1-day bars of contracts for the duration specified
    Args:
        (contract) as obj
        (duration) as int
    Returns: dataframe of symbol, date, ohlc, avg and volume 
    '''
    
    # Prepare the duration
    strduration = str(duration) + ' D'
    
    # Extract the history
    hist = ib.reqHistoricalData(contract=contract, endDateTime='', 
                                    durationStr=strduration, barSizeSetting='1 day',  
                                                whatToShow='Trades', useRTH=True)
    
    df = util.df(hist)
    df.insert(0, column='symbol', value=contract.symbol)
    
    return df

#...function to get price and dividend ticker
def get_dividend_ticker(contract):
    '''Gets dividend ticker of the contract
    Arg: (contract) as a qualified contract object with conId
    Returns: ticker'''
    
    ib.reqMktData(contract, '456', snapshot=False, regulatorySnapshot=False) # request ticker stream

    ticker = ib.ticker(contract)
    
    # Ensure the ticker is filled
    while ticker.dividends is None:
        while np.isnan(ticker.marketPrice()):
            ib.sleep(1)

    ib.cancelMktData(contract)
       
    return ticker

def grp_opts(df):
    '''Groups options and sorts strikes by puts and calls
    Arg: df as dataframe
    Returns: sorted dataframe'''
    
    gb = df.groupby('right')

    if 'C' in [k for k in gb.indices]:
        df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, True])
    else:
        df_calls =  pd.DataFrame([])

    if 'P' in [k for k in gb.indices]:
        df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, False])
    else:
        df_puts =  pd.DataFrame([])

    df = pd.concat([df_puts, df_calls]).reset_index(drop=True)
    
    return df

def filter_kxdte(df, df_ohlc):
    '''Filters the strikes by dte*3, and, min of lows for Puts and max of highs for Calls
    Arg: df as dataframe
    Returns: cleansed dfs without the risky strikes'''
    
    df['sfilt'] = [df_ohlc.set_index('symbol').loc[s][:max(ohlc_min_dte, d*3)].low.min() 
     if g == 'P' 
     else df_ohlc.set_index('symbol').loc[s][:max(ohlc_min_dte, d*3)].high.max() 
     for s, d, g in zip(df.symbol, df.dte, df.right)]
    
    df = df[((df.right == 'P') & (df.strike < df.sfilt)) | \
            ((df.right == 'C') & (df.strike > df.sfilt))]
    
    # return sorted df of puts and calls    
    df1 = grp_opts(df)
    
    # drop sfilt    
    return df1.drop('sfilt', axis=1)

#... Black-Scholes
# Ref: - https://ideone.com/fork/XnikMm - Brian Hyde

def get_bsm(undPrice, strike, dte, rate, volatility, divrate):
    ''' Gets Black Scholes output
    Args:
        (undPrice) : Current Stock Price in float
        (strike)   : Strike Price in float
        (dte)      : Days to expiration in float
        (rate)     : dte until expiry in days
        (volatility)    : Standard Deviation of stock's return in float
        (divrate)  : Dividend Rate in float
    Returns:
        (delta, call_price, put_price) as a tuple
    '''
    #statistics
    sigTsquared = sqrt(dte/365)*volatility
    edivT = exp((-divrate*dte)/365)
    ert = exp((-rate*dte)/365)
    d1 = (log(undPrice*edivT/strike)+(rate+.5*(volatility**2))*dte/365)/sigTsquared
    d2 = d1-sigTsquared
    Nd1 = (1+erf(d1/sqrt(2)))/2
    Nd2 = (1+erf(d2/sqrt(2)))/2
    iNd1 = (1+erf(-d1/sqrt(2)))/2
    iNd2 = (1+erf(-d2/sqrt(2)))/2

    #Outputs
    callPrice = round(undPrice*edivT*Nd1-strike*ert*Nd2, 2)
    putPrice = round(strike*ert*iNd2-undPrice*edivT*iNd1, 2)
    delta = Nd1

    return {'bsmCall': callPrice, 'bsmPut': putPrice, 'bsmDelta': delta}

#... assignments
#_______________
exchange = 'NSE'
fspath = './zdata/'

maxdte = 65  # max expiry date for options
mindte = 1  # min expiry date for options

ohlc_min_dte = 45     # no of minimum dte days to determine strikes based on ohlc

tradingdays = 252

blks = 50

#... Get risk-free rate from 91 day T-bills
rate_url = 'https://rbi.org.in/home.aspx'

li = pd.read_html(rate_url)
li_df = li[4].rename(columns = {0: 'Cat', 1: 'Values'})
li_val = li_df.loc[li_df.Cat == '91 day T-bills', 'Values']
rate = float((str(li_val).split('\n')[0].split('%')[0].split(' ')[-1:])[0])/100

#... build the symbols
#______________________

tp = pd.read_html('https://www.tradeplusonline.com/Equity-Futures-Margin-Calculator.aspx')

df_tp = tp[1][2:].iloc[:, :-1]
df_tp = df_tp.iloc[:, [0,1,5]]
df_tp.columns=['nseSymbol', 'lot', 'margin']

cols = df_tp.columns.drop('nseSymbol')
df_tp[cols] = df_tp[cols].apply(pd.to_numeric, errors='coerce') # convert lot and margin to numeric

df_slm = df_tp.copy()

# Truncate to 9 characters for ibSymbol
df_slm['ibSymbol'] = df_slm.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_slm.ibSymbol = df_slm.ibSymbol.replace(ntoi)

# separate indexes and equities, eliminate discards from df_slm
indexes = ['NIFTY50', 'BANKNIFTY']
discards = ['NIFTYMID5', 'NIFTYIT', 'LUPIN']
equities = sorted([s for s in df_slm.ibSymbol if s not in indexes+discards])

symbols = equities+indexes

cs = [Stock(s, exchange) if s in equities else Index(s, exchange) for s in symbols]

qcs = ib.qualifyContracts(*cs) # qualified underlyings

#****           Single scrip check. To be DELETED in function          *****
#...........................................................................
contract = next(q for q in qcs if q.symbol=='ACC')  # one symbol logic check
#___________________________________________________________________________

def get_pkl(contract):
    #... get ohlc, with cumulative volatality and standard deviation
    #_______________________________________________________________

    df_ohlc = get_hist(contract, 365).set_index('date').sort_index(ascending = False)

    # get cumulative standard deviation
    df_stdev = pd.DataFrame(df_ohlc['close'].expanding(1).std(ddof=0))
    df_stdev.columns = ['stdev']

    # get cumulative volatility
    df_vol = pd.DataFrame(df_ohlc['close'].pct_change().expanding(1).std(ddof=0)*sqrt(tradingdays))
    df_vol.columns = ['volatility']

    df_ohlc1 = df_ohlc.join(df_vol)

    df_ohlc2 = df_ohlc1.join(df_stdev)

    #pickle the ohlc
    df_ohlc2.to_pickle(fspath+contract.symbol+'_ohlc.pkl')

    #... get the underlyings
    #_______________________

    ticker = get_dividend_ticker(contract)

    df_und = util.df([ticker])

    cols = ['contract', 'time', 'bid', 'bidSize', 'ask', 'askSize', 'last', 'lastSize', 
            'volume', 'open', 'high', 'low', 'close', 'dividends']
    df_und = df_und[cols]

    df_und = df_und.assign(undPrice=np.where(df_und['last'].isnull(), df_und.close, df_und['last']))

    try: 
        divrate = df_und.dividends[0][0]/df_und.dividends[0][0]/df_und.undPrice
    except (TypeError, AttributeError) as e:
        divrate = 0.0

    df_und = df_und.assign(divrate=divrate)

    df_und = df_und.assign(symbol=[c[1].symbol for c in df_und.contract.items()])

    #... get the lot, margin, undPrice and dividend rate
    undlot = df_slm.loc[df_slm.ibSymbol == contract.symbol, 'lot'].item()
    df_und['lot'] = undlot

    # margin of underlying
    order = Order(action='SELL', totalQuantity=undlot, orderType='MKT')

    # margin = float(ib.whatIfOrder(contract, order).initMarginChange)
    margin = df_slm.loc[df_slm.ibSymbol == contract.symbol, 'margin'].item()

    df_und['margin'] = margin

    df_und.to_pickle(fspath+contract.symbol+'_und.pkl')

    #... get the options
    #___________________

    # symbol
    symbol = contract.symbol

    # rights
    right = ['P', 'C'] 

    # chains
    chains = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, futFopExchange='', 
                          underlyingConId=contract.conId, underlyingSecType=contract.secType)

    chain = next(c for c in chains if c.exchange == exchange)

    undPrice = df_und.undPrice[0]

    # limit the expirations to between min and max dates
    expirations = sorted([exp for exp in chain.expirations 
                          if mindte < get_dte(exp) < maxdte])

    # get the strikes
    strikes = sorted([strike for strike in chain.strikes])

    # build the puts and calls
    rights = ['P', 'C']

    df_tgt = pd.DataFrame([i for i in itertools.product([symbol], expirations, strikes, rights)], columns=['symbol', 'expiry', 'strike', 'right'])
    df_tgt['dte'] = [get_dte(e) for e in df_tgt.expiry]

    df_tgt1 = filter_kxdte(df_tgt, df_ohlc2)

    # make the contracts
    contracts = [Option(symbol, expiry, strike, right, exchange) 
                 for symbol, expiry, strike, right 
                 in zip(df_tgt1.symbol, df_tgt1.expiry, df_tgt1.strike, df_tgt1.right)]

    qc = [ib.qualifyContracts(*contracts[i: i+blks]) for i in range(0, len(contracts), blks)]

    qc1 = [q for q1 in qc for q in q1]
    df_qc = util.df(qc1).iloc[:, [2,3,4,5]]
    df_qc.columns=['symbol', 'expiry', 'strike', 'right']

    df_opt = df_qc.merge(df_tgt, on=list(df_qc), how='inner')
    df_opt['option'] = qc1

    df_und1 = df_und[['symbol', 'undPrice', 'lot', 'margin']].set_index('symbol') # get respective columns from df_und

    df_opt = df_opt.set_index('symbol').join(df_und1) # join for lot and margin

    # get the standard deviation based on days to expiry
    df_opt = df_opt.assign(stdev=[df_ohlc2.iloc[i].stdev for i in df_opt.dte])

    # get the volatality based on days to expiry
    df_opt = df_opt.assign(volatility=[df_ohlc2.iloc[i].volatility for i in df_opt.dte])

    # high52 and low52 for the underlying
    df_opt = df_opt.assign(hi52 = df_ohlc2[:252].high.max())
    df_opt = df_opt.assign(lo52 = df_ohlc2[:252].low.min())
    df_opt.loc[df_opt.right == 'P', 'hi52'] = np.nan
    df_opt.loc[df_opt.right == 'C', 'lo52'] = np.nan

    df_opt.loc[df_opt.dte <= 1, 'dte'] = 2 # Make the dte as 2 for 1 day-to-expiry to prevent bsm divide-by-zero error

    # get the black scholes delta, call and put prices
    bsms = [get_bsm(undPrice, strike, dte, rate, volatility, divrate) 
            for undPrice, strike, dte, rate, volatility, divrate in 
            zip(itertools.repeat(undPrice), df_opt.strike, df_opt.dte, itertools.repeat(rate), df_opt.volatility, itertools.repeat(divrate))]

    df_bsm = pd.DataFrame(bsms)

    df_opt = df_opt.reset_index().join(df_bsm) # join with black-scholes

    df_opt['bsmPrice'] = np.where(df_opt.right == 'P', df_opt.bsmPut, df_opt.bsmCall)
    df_opt['pop'] = np.where(df_opt.right == 'C', 1-df_opt.bsmDelta, df_opt.bsmDelta)
    df_opt = df_opt.drop(['bsmCall', 'bsmPut', 'bsmDelta'], axis=1)

    # get the option prices
    cs = list(df_opt.option)

    # [catch(lambda: ib.reqTickers(c).marketPrice()) for i in range(0, len(cs), 100) for c in cs[i: i+100]]
    tickers = [ib.reqTickers(*cs[i: i+100]) for i in range(0, len(cs), 100)]

    df_opt = df_opt.assign(price=[t.marketPrice() for ts in tickers for t in ts])

    df_opt = df_opt.assign(rom=df_opt.price/df_opt.margin*tradingdays/df_opt.dte*df_opt.lot)

    df_opt.to_pickle(fspath+contract.symbol+'_opt.pkl')
    
    return None

Started to throttle requests
Stopped to throttle requests


Wall time: 34.6 s


In [3]:
%%time

symbols = [s.symbol for s in qcs]

# Pickle the dataframes
fspath = './zdata/'

# Take only pickle files. Remove directories and files starting with underscore (for underlyings)
fs = [f for f in os.listdir(fspath) if (f[-7:] == 'opt.pkl')]

# If the path is empty, start filling it
# Else start from where you left!
if not fs:
    
    [catch(lambda: get_pkl(t)) for t in qcs]
    
    ib.disconnect()
    
else:
    # Get modified time, fail time and identify where the scrip has failed
    fsmod = {f: os.path.getmtime(fspath + '/' + f) for f in fs}

    failtime = max([v for k, v in fsmod.items()])

    failscrip = [k[:-4] for k, v in fsmod.items() if v == failtime][0]    

    restartfrom = symbols.index(failscrip[:-4]) + 1

    xs = [f[:-8] for f in fs] # existing symbols

    rc = [c for c in qcs if c.symbol not in xs] # remaining qcs

    # Restart the pickling!
    [catch(lambda: get_pkl(t)) for t in rc]
    
    ib.disconnect()

Started to throttle requests
Stopped to throttle requests
Error 200, reqId 266: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20190425', strike=1860.0, right='C', exchange='NSE')
Error 200, reqId 268: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20190425', strike=1880.0, right='C', exchange='NSE')
Error 200, reqId 264: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20190425', strike=1840.0, right='C', exchange='NSE')
Error 200, reqId 270: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20190425', strike=1900.0, right='C', exchange='NSE')
Error 200, reqId 272: No security definition has been found for the request, contract: Option(symbol='ACC', lastTradeDateOrContractMonth='20190425', strike=1920.0, right='C', exc

Error 200, reqId 446: No security definition has been found for the request, contract: Option(symbol='ADANIPORT', lastTradeDateOrContractMonth='20190425', strike=465.0, right='C', exchange='NSE')
Error 200, reqId 449: No security definition has been found for the request, contract: Option(symbol='ADANIPORT', lastTradeDateOrContractMonth='20190328', strike=475.0, right='C', exchange='NSE')
Error 200, reqId 450: No security definition has been found for the request, contract: Option(symbol='ADANIPORT', lastTradeDateOrContractMonth='20190425', strike=475.0, right='C', exchange='NSE')
Error 200, reqId 462: No security definition has been found for the request, contract: Option(symbol='ADANIPORT', lastTradeDateOrContractMonth='20190425', strike=530.0, right='C', exchange='NSE')
Error 200, reqId 464: No security definition has been found for the request, contract: Option(symbol='ADANIPORT', lastTradeDateOrContractMonth='20190425', strike=540.0, right='C', exchange='NSE')
Error 200, reqId 466

Error 200, reqId 973: No security definition has been found for the request, contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20190425', strike=105.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20190328', strike=105.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20190425', strike=105.0, right='P', exchange='NSE')
Error 200, reqId 1002: No security definition has been found for the request, contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20190425', strike=310.0, right='C', exchange='NSE')
Error 200, reqId 1004: No security definition has been found for the request, contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20190425', strike=315.0, right='C', exchange='NSE')
Error 200, reqId 1005: No security definition has been found for the request, contract: Option(symbol='AMBUJACEM', lastTradeDateOrContractMonth='20190328', 

Unknown contract: Option(symbol='ARVIND', lastTradeDateOrContractMonth='20190328', strike=200.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='ARVIND', lastTradeDateOrContractMonth='20190328', strike=210.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='ARVIND', lastTradeDateOrContractMonth='20190328', strike=220.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='ARVIND', lastTradeDateOrContractMonth='20190328', strike=230.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='ARVIND', lastTradeDateOrContractMonth='20190328', strike=240.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='ARVIND', lastTradeDateOrContractMonth='20190328', strike=250.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='ARVIND', lastTradeDateOrContractMonth='20190328', strike=260.0, right='C', exchange='NSE')
Error 200, reqId 1316: No security definition has been found for the request, contract: Option(symbol='ASHOKLEY', lastTradeDateOrCon

Unknown contract: Option(symbol='ASIANPAIN', lastTradeDateOrContractMonth='20190425', strike=900.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='ASIANPAIN', lastTradeDateOrContractMonth='20190328', strike=880.0, right='P', exchange='NSE')
Unknown contract: Option(symbol='ASIANPAIN', lastTradeDateOrContractMonth='20190425', strike=880.0, right='P', exchange='NSE')
Error 200, reqId 1569: No security definition has been found for the request, contract: Option(symbol='AUROPHARM', lastTradeDateOrContractMonth='20190328', strike=1140.0, right='C', exchange='NSE')
Error 200, reqId 1570: No security definition has been found for the request, contract: Option(symbol='AUROPHARM', lastTradeDateOrContractMonth='20190425', strike=1140.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='AUROPHARM', lastTradeDateOrContractMonth='20190328', strike=1140.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='AUROPHARM', lastTradeDateOrContractMonth='20190425', strike=11

...

Error 200, reqId 20662: No security definition has been found for the request, contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190307', strike=13500.0, right='C', exchange='NSE')
Error 200, reqId 20664: No security definition has been found for the request, contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190320', strike=13500.0, right='C', exchange='NSE')
Error 200, reqId 20666: No security definition has been found for the request, contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190404', strike=13500.0, right='C', exchange='NSE')
Error 200, reqId 20667: No security definition has been found for the request, contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190411', strike=13500.0, right='C', exchange='NSE')
Error 200, reqId 20668: No security definition has been found for the request, contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190418', strike=13500.0, right='C', exchange='NSE')
Error 200,

Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190425', strike=13400.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190502', strike=13400.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190307', strike=13500.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190314', strike=13500.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190320', strike=13500.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190404', strike=13500.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190411', strike=13500.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190418', strike=13500.0, right='C', 

Error 200, reqId 20724: No security definition has been found for the request, contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190502', strike=14100.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190320', strike=14000.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190404', strike=14000.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190411', strike=14000.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190418', strike=14000.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190425', strike=14000.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50', lastTradeDateOrContractMonth='20190502', strike=14000.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='NIFTY50

Unknown contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190418', strike=29000.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190502', strike=29000.0, right='C', exchange='NSE')
Unknown contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190404', strike=29100.0, right='C', exchange='NSE')
Error 200, reqId 21297: No security definition has been found for the request, contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190411', strike=29100.0, right='C', exchange='NSE')
Error 200, reqId 21298: No security definition has been found for the request, contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190418', strike=29100.0, right='C', exchange='NSE')
Error 200, reqId 21299: No security definition has been found for the request, contract: Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth='20190425', strike=29100.0, right='C', exchange='NSE')
Error 200, re

Wall time: 1h 10min 57s


In [ ]:
ib.disconnect()
ib.sleep(4)
ib = IB().connect('127.0.0.1', 3000, clientId=1)